In [3]:
%load_ext autoreload
%autoreload 2
notebook_fixed_dir = False

In [4]:
# this cell can only be called once
import os
if not notebook_fixed_dir:
    os.chdir('..')
    notebook_fixed_dir = True
print(os.getcwd())

/home/svcl-oowl/brandon/research/sil_consistent_at_inference


In [5]:
# imports
import pprint
import pickle
import glob
import random
from pathlib import Path

import torch
from tqdm import tqdm
from PIL import Image
import numpy as np
from pytorch3d.renderer import (
    look_at_view_transform
)
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import numpy as np

from utils import utils
from utils import eval_utils
import deformation.losses as def_losses


In [9]:
#evaluation_yaml_path = "data_evaluation/shapenet_chair/occnet.yaml"
#evaluation_yaml_path = "data_evaluation/shapenet_chair/occnet_refined.yaml"

evaluation_yaml_path = "data_evaluation/pix3d_chair/occnet_refined.yaml"
#evaluation_yaml_path = "data_evaluation/pix3d_chair/occnet.yaml"

results_df = pd.read_pickle(evaluation_yaml_path.replace(".yaml","_eval_results.pkl"))
print(evaluation_yaml_path)
display(results_df)
print(results_df.mean())

data_evaluation/pix3d_chair/occnet_refined.yaml


,2d_iou,3d_iou,chamfer_L1,gt_obj_path,input_img_path,instance,rec_obj_path
0,0.398359,0.051361,0.024908,/home/svcl-oowl/dataset/pix3d/model/chair/SS_0...,/home/svcl-oowl/brandon/research/sil_consisten...,0191,/home/svcl-oowl/brandon/research/sil_consisten...
1,0.716822,0.025357,0.014278,/home/svcl-oowl/dataset/pix3d/model/chair/SS_0...,/home/svcl-oowl/brandon/research/sil_consisten...,0225,/home/svcl-oowl/brandon/research/sil_consisten...
2,0.197343,0.082071,0.098394,/home/svcl-oowl/dataset/pix3d/model/chair/IKEA...,/home/svcl-oowl/brandon/research/sil_consisten...,0103,/home/svcl-oowl/brandon/research/sil_consisten...
3,0.616212,0.112327,0.075430,/home/svcl-oowl/dataset/pix3d/model/chair/SS_0...,/home/svcl-oowl/brandon/research/sil_consisten...,0649,/home/svcl-oowl/brandon/research/sil_consisten...
4,0.720145,0.133940,0.013872,/home/svcl-oowl/dataset/pix3d/model/chair/SS_0...,/home/svcl-oowl/brandon/research/sil_consisten...,0699,/home/svcl-oowl/brandon/research/sil_consisten...
...,...,...,...,...,...,...,...
2819,0.264988,0.108234,0.051982,/home/svcl-oowl/dataset/pix3d/model/chair/IKEA...,/home/svcl-oowl/brandon/research/sil_consisten...,3131,/home/svcl-oowl/brandon/research/sil_consisten...
2820,0.399162,0.039422,0.010355,/home/svcl-oowl/dataset/pix3d/model/chair/IKEA...,/home/svcl-oowl/brandon/research/sil_consisten...,3189,/home/svcl-oowl/brandon/research/sil_consisten...
2821,0.331584,0.034754,0.017136,/home/svcl-oowl/dataset/pix3d/model/chair/IKEA...,/home/svcl-oowl/brandon/research/sil_consisten...,3712,/home/svcl-oowl/brandon/research/sil_consisten...
2822,0.554682,0.021351,0.041058,/home/svcl-oowl/dataset/pix3d/model/chair/IKEA...,/home/svcl-oowl/brandon/research/sil_consisten...,3275,/home/svcl-oowl/brandon/research/sil_consisten...


2d_iou        0.555858
3d_iou        0.124136
chamfer_L1    0.197908
instance           inf
dtype: float64


In [10]:
def compare_before_after_refinement(yaml_path_before, yaml_path_after):
    results_df_before = pd.read_pickle(yaml_path_before.replace(".yaml","_eval_results.pkl"))
    results_df_after = pd.read_pickle(yaml_path_after.replace(".yaml","_eval_results.pkl"))
    
    statistics = {"num_3d_iou_improved":0, "avg_improve_amt": 0, "avg_improve_before_2d_iou":0, "num_3d_iou_worse":0,  "avg_worse_amt":0, "avg_worse_before_2d_iou":0}
    improved_instances = []
    worse_instances = []
    for instance in list(results_df_after["instance"]):
        before_result = results_df_before[results_df_before["instance"]==instance]
        before_3d_iou = float(before_result["3d_iou"])
        before_2d_iou = float(before_result["2d_iou"])
        after_result = results_df_after[results_df_after["instance"]==instance]
        after_3d_iou = float(after_result["3d_iou"])
        after_2d_iou = float(after_result["2d_iou"])
        
        iou_2d_change = after_2d_iou - before_2d_iou
        iou_3d_change = after_3d_iou - before_3d_iou
        
        if iou_3d_change > 0:
            statistics["num_3d_iou_improved"] += 1
            statistics["avg_improve_amt"] += iou_3d_change
            statistics["avg_improve_before_2d_iou"] += before_2d_iou
            improved_instances.append(instance)
        else:
            statistics["num_3d_iou_worse"] += 1
            statistics["avg_worse_amt"] += iou_3d_change
            statistics["avg_worse_before_2d_iou"] += before_2d_iou
            worse_instances.append(instance)
    
    statistics["avg_improve_amt"] /= statistics["num_3d_iou_improved"]
    statistics["avg_improve_before_2d_iou"] /= statistics["num_3d_iou_improved"]
    statistics["avg_worse_amt"] /= statistics["num_3d_iou_worse"]
    statistics["avg_worse_before_2d_iou"] /= statistics["num_3d_iou_worse"]
            
    pprint.pprint(statistics)
    print("example improved instances")
    print(improved_instances[:100])
    print("example worse instances")
    print(worse_instances[:100])

In [11]:
dataset = "pix3d"
evaluation_yaml_path_before = "data_evaluation/{}_chair/occnet.yaml".format(dataset)
evaluation_yaml_path_after = "data_evaluation/{}_chair/occnet_refined.yaml".format(dataset)
compare_before_after_refinement(evaluation_yaml_path_before, evaluation_yaml_path_after)

{'avg_improve_amt': 0.05432854915685779,
 'avg_improve_before_2d_iou': 0.5134864536233673,
 'avg_worse_amt': -0.1112035454593573,
 'avg_worse_before_2d_iou': 0.5005666220267182,
 'num_3d_iou_improved': 872,
 'num_3d_iou_worse': 1961}
example improved instances
['0103', '0701', '0073', '0069', '0607', '0681', '0208', '0757', '0254', '0166', '0514', '0767', '0292', '0529', '0356', '0702', '0318', '0625', '0596', '0031', '0606', '0067', '0552', '0246', '0068', '0435', '0426', '0632', '0689', '0535', '0741', '0258', '0631', '0139', '0171', '0104', '0709', '0144', '0413', '0233', '0719', '0748', '0411', '0336', '0065', '0398', '0619', '0597', '0329', '0380', '0707', '0553', '0494', '0743', '0043', '0588', '0305', '0647', '0252', '0271', '0150', '0509', '0014', '0708', '0490', '0684', '0573', '0167', '0558', '0763', '0396', '0752', '0201', '0412', '0543', '0125', '0070', '0484', '0766', '0691', '0546', '0064', '0034', '0710', '0234', '0353', '0533', '0638', '0549', '0592', '0075', '0485', '0

In [ ]:
# 0356, 0702, 0596, 0281, 0737
# 0085, 0372, 0151, 0409, 0578